In [2]:
import pandas as pd
import subprocess
import json
import sys
import requests
from lxml import etree


pd.set_option('max_colwidth',250)
pd.set_option('display.max_rows',90)
# sys.setdefaultencoding('utf-8')

In [3]:
def print_tags(element, indent=0):
        print("  " * indent + element.tag)
        for child in element:
            print_tags(child, indent + 1)

def tabelaParlamentar(parlamentares):
    tabela = []
    for parlamentar in parlamentares:
        treeParlamentar = etree.fromstring(etree.tostring(parlamentar, encoding="unicode"))
        try:
            partido = treeParlamentar.xpath('//IdentificacaoParlamentar//SiglaPartidoParlamentar/text()')[0]
        except:
            partido = "N/I"

        try:
            codigoBloco = treeParlamentar.xpath('//IdentificacaoParlamentar//Bloco//CodigoBloco/text()')[0]
            NomeBloco = treeParlamentar.xpath('//IdentificacaoParlamentar//Bloco//NomeBloco/text()')[0]
            NomeApelido = treeParlamentar.xpath('//IdentificacaoParlamentar//Bloco//NomeApelido/text()')[0]
            DataCriacao = treeParlamentar.xpath('//IdentificacaoParlamentar//Bloco//DataCriacao/text()')[0],
        except:
            codigoBloco = "N/I"
            NomeBloco = "N/I"
            NomeApelido = "N/I"
            DataCriacao = "N/I"

        tabela.append([
            treeParlamentar.xpath('//IdentificacaoParlamentar//CodigoParlamentar/text()')[0],
            treeParlamentar.xpath('//IdentificacaoParlamentar//CodigoPublicoNaLegAtual/text()')[0],
            treeParlamentar.xpath('//IdentificacaoParlamentar//NomeParlamentar/text()')[0],
            treeParlamentar.xpath('//IdentificacaoParlamentar//NomeCompletoParlamentar/text()')[0],
            treeParlamentar.xpath('//IdentificacaoParlamentar//SexoParlamentar/text()')[0],
            treeParlamentar.xpath('//IdentificacaoParlamentar//FormaTratamento/text()')[0],
            treeParlamentar.xpath('//IdentificacaoParlamentar//UrlFotoParlamentar/text()')[0],
            treeParlamentar.xpath('//IdentificacaoParlamentar//UrlPaginaParlamentar/text()')[0],
            treeParlamentar.xpath('//IdentificacaoParlamentar//EmailParlamentar/text()')[0],
            partido,
            treeParlamentar.xpath('//IdentificacaoParlamentar//UfParlamentar/text()')[0],
            codigoBloco,
            NomeBloco,
            NomeApelido,
            DataCriacao,
            treeParlamentar.xpath('//IdentificacaoParlamentar//MembroMesa/text()')[0],
            treeParlamentar.xpath('//IdentificacaoParlamentar//MembroLideranca/text()')[0],
        ])
    return tabela

def api(api_url):
    response = requests.get(api_url)
    if response.status_code == 200:
        return (response.content)
    else:
        print(f"Error: {response.status_code}")

def retornarTree(api_url):
    string = api(api_url)
    tree = etree.fromstring(string)
    root = tree.tag
    return [tree,root]

def parlamentarMain():
    api_url = 'https://legis.senado.leg.br/dadosabertos/senador/lista/atual'
    tree = retornarTree(api_url)
    root = tree[1]
    tree = tree[0]
    parlamentares = tree.xpath('//ListaParlamentarEmExercicio//Parlamentares//Parlamentar')
    colunas = ["CodigoParlamentar","CodigoPublicoNaLegAtual","NomeParlamentar","NomeCompletoParlamentar","SexoParlamentar","FormaTratamento","UrlFotoParlamentar","UrlPaginaParlamentar","EmailParlamentar","SiglaPartidoParlamentar","UfParlamentar","CodigoBloco","NomeBloco","NomeApelido","DataCriacao","MembroMesa","MembroLideranca",]
    tabelaParlamentares = tabelaParlamentar(parlamentares)
    salvarArquivo(tabelaParlamentares,"parlamentares",colunas)
    return pd.DataFrame(tabelaParlamentares, columns=colunas)

def salvarArquivo(lista,nome,colunas):
    dados = pd.DataFrame(lista, columns=colunas)
    dados.to_csv("../arquivos/"+nome+".csv")

In [4]:
parlamentares = parlamentarMain()

In [5]:
def listaAutoriasParlamentares(Autorias,CodigoParlamentar):
    tabela = []
    for Autoria in Autorias:
        treeAutoria = etree.fromstring(etree.tostring(Autoria, encoding="unicode"))
        try:
            IndicadorOutrosAutores = treeAutoria.xpath('//Autoria//IndicadorOutrosAutores/text()')[0]
        except:
            IndicadorOutrosAutores = "N/I"
        try:
            Ementa = treeAutoria.xpath('//Materia//Ementa/text()')[0]
        except:
            Ementa = "N/I"
        tabela.append([
            CodigoParlamentar,
            treeAutoria.xpath('//Materia//Codigo/text()')[0],
            treeAutoria.xpath('//Materia//IdentificacaoProcesso/text()')[0],
            treeAutoria.xpath('//Materia//DescricaoIdentificacao/text()')[0],
            treeAutoria.xpath('//Materia//Sigla/text()')[0],
            treeAutoria.xpath('//Materia//Numero/text()')[0],
            treeAutoria.xpath('//Materia//Ano/text()')[0],
            Ementa,
            treeAutoria.xpath('//Materia//Data/text()')[0],
            IndicadorOutrosAutores
        ])
    return tabela

def AutoriasPorParlamentar(CodigoParlamentar):
    api_url = 'https://legis.senado.leg.br/dadosabertos/senador/'+CodigoParlamentar+'/autorias'
    tree = retornarTree(api_url)
    root = tree[1]
    tree = tree[0]
    Autorias = tree.xpath('//MateriasAutoriaParlamentar//Parlamentar//Autorias//Autoria')
    listaAutorias = listaAutoriasParlamentares(Autorias,CodigoParlamentar)
    return listaAutorias
def autoriasMain():
    autoriasTEMP = []
    for i,r in parlamentares.iterrows():
        autoriasTEMP.append(AutoriasPorParlamentar(r[0]))
    autorias = []
    for i in range(len(autoriasTEMP)):
        for j in range(len(autoriasTEMP[i])):
            autorias.append(autoriasTEMP[i][j])
    autoriasTEMP = None
    colunas = ["CodigoParlamentar","CodigoMateria","IdentificacaoProcesso","DescricaoIdentificacao","Sigla","Numero","Ano","Ementa","Data","IndicadorOutrosAutores",]
    autorias = pd.DataFrame(autorias,columns=colunas)
    salvarArquivo(autorias,"autorias",colunas)
    return autorias

In [6]:
autorias = autoriasMain()

C:\Users\krist\AppData\Local\Temp\ipykernel_17112\4149234536.py:38: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  autoriasTEMP.append(AutoriasPorParlamentar(r[0]))


In [106]:
def votacoesPorParlamentar(CodigoParlamentar):
    api_url = 'https://legis.senado.leg.br/dadosabertos/senador/'+CodigoParlamentar+'/votacoes'
    tree = retornarTree(api_url)
    root = tree[1]
    tree = tree[0]
    Votacoes = tree.xpath('//VotacaoParlamentar//Parlamentar//Votacoes//Votacao')
    return Votacoes
    # listaAutorias = listaAutoriasParlamentares(Autorias,CodigoParlamentar)

def listaVotacoesParlamentares(votacoes,codigoSenador):
    tabela = []
    for voto in votacoes:
        treeVoto = etree.fromstring(etree.tostring(voto, encoding="unicode"))
        CodigoSessao = treeVoto.xpath('//SessaoPlenaria//CodigoSessao/text()')[0]
        CasaSessao = treeVoto.xpath('//SessaoPlenaria//SiglaCasaSessao/text()')[0]
        DataSessao = treeVoto.xpath('//SessaoPlenaria//DataSessao/text()')[0]
        CodigoMateria = treeVoto.xpath('//Materia//Codigo/text()')[0]
        voto = treeVoto.xpath('//SiglaDescricaoVoto/text()')[0]
        VotacaoSecreta = treeVoto.xpath('//IndicadorVotacaoSecreta/text()')[0]
        print(codigoSenador,CodigoSessao,CasaSessao,DataSessao,CodigoMateria,voto,VotacaoSecreta)
        tabela.append([
            codigoSenador,
            CodigoSessao,
            CasaSessao,
            DataSessao,
            CodigoMateria,
            voto,
            VotacaoSecreta,
        ])
    return tabela
    

In [107]:
votacoes = votacoesPorParlamentar('1')
listaVotacoesParlamentares(votacoes,'1')
# print_tags(votacoes)

1 4722 SF 2000-05-10 41441 P-NRV Não
1 6587 SF 2001-10-18 45483 Não Não
1 6567 SF 2001-10-17 40941 Não Não
1 6567 SF 2001-10-17 40941 Sim Não
1 6567 SF 2001-10-17 1546 Sim Não
1 6548 SF 2001-10-16 48580 Sim Não
1 6507 SF 2001-10-10 41526 Sim Não
1 6488 SF 2001-10-09 813 Sim Não
1 6427 SF 2001-10-02 1552 P-NRV Não
1 6427 SF 2001-10-02 1552 P-NRV Não
1 6287 SF 2001-09-18 43730 Sim Não
1 6223 SF 2001-09-05 47830 Sim Não
1 6222 SF 2001-09-04 40987 Sim Não
1 6181 SF 2001-08-29 44093 P-NRV Não
1 6181 SF 2001-08-29 44093 P-NRV Não
1 6181 SF 2001-08-29 46823 P-NRV Não
1 6021 SF 2001-08-15 818 Sim Não
1 6021 SF 2001-08-15 47830 Sim Não
1 6021 SF 2001-08-15 44443 Sim Não
1 6021 SF 2001-08-15 44443 Sim Não
1 5961 SF 2001-08-08 41050 P-NRV Não
1 5860 SF 2001-06-27 44443 Sim Não
1 5860 SF 2001-06-27 44093 Sim Não
1 5840 SF 2001-06-26 47377 Sim Não
1 5840 SF 2001-06-26 47377 Sim Não
1 5782 SF 2001-06-20 45834 P-NRV Não
1 5782 SF 2001-06-20 45834 P-NRV Não
1 5782 SF 2001-06-20 47389 P-NRV Não
1 5737 

[['1', '4722', 'SF', '2000-05-10', '41441', 'P-NRV', 'Não'],
 ['1', '6587', 'SF', '2001-10-18', '45483', 'Não', 'Não'],
 ['1', '6567', 'SF', '2001-10-17', '40941', 'Não', 'Não'],
 ['1', '6567', 'SF', '2001-10-17', '40941', 'Sim', 'Não'],
 ['1', '6567', 'SF', '2001-10-17', '1546', 'Sim', 'Não'],
 ['1', '6548', 'SF', '2001-10-16', '48580', 'Sim', 'Não'],
 ['1', '6507', 'SF', '2001-10-10', '41526', 'Sim', 'Não'],
 ['1', '6488', 'SF', '2001-10-09', '813', 'Sim', 'Não'],
 ['1', '6427', 'SF', '2001-10-02', '1552', 'P-NRV', 'Não'],
 ['1', '6427', 'SF', '2001-10-02', '1552', 'P-NRV', 'Não'],
 ['1', '6287', 'SF', '2001-09-18', '43730', 'Sim', 'Não'],
 ['1', '6223', 'SF', '2001-09-05', '47830', 'Sim', 'Não'],
 ['1', '6222', 'SF', '2001-09-04', '40987', 'Sim', 'Não'],
 ['1', '6181', 'SF', '2001-08-29', '44093', 'P-NRV', 'Não'],
 ['1', '6181', 'SF', '2001-08-29', '44093', 'P-NRV', 'Não'],
 ['1', '6181', 'SF', '2001-08-29', '46823', 'P-NRV', 'Não'],
 ['1', '6021', 'SF', '2001-08-15', '818', 'Sim', 

In [23]:
aut = votacoesPorParlamentar('1')
aut

VotacaoParlamentar
  Metadados
    Versao
    VersaoServico
    DataVersaoServico
    DescricaoDataSet
  Parlamentar
    Codigo
    Nome
    Votacoes
      Votacao
        SessaoPlenaria
          CodigoSessao
          SiglaCasaSessao
          CodigoSessaoLegislativa
          SiglaTipoSessao
          NumeroSessao
          DataSessao
          HoraInicioSessao
        Materia
          Codigo
          IdentificacaoProcesso
          DescricaoIdentificacao
          Sigla
          Numero
          Ano
          Ementa
          Data
        Tramitacao
          IdentificacaoTramitacao
            CodigoTramitacao
            NumeroAutuacao
            DataTramitacao
            TextoTramitacao
            OrigemTramitacao
              Local
                CodigoLocal
                TipoLocal
                SiglaCasaLocal
                SiglaLocal
                NomeLocal
            DestinoTramitacao
              Local
                CodigoLocal
                TipoLocal
 